In [34]:
import torch
import sklearn
import numpy as np
import pandas as pd
from tqdm import tqdm

In [35]:
train = pd.read_csv("dataset/random_instruction_train.csv")

In [25]:
train

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label
0,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0,B
1,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1,C
2,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0,A
3,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0,B
4,292873,vicuna-13b,gpt-4-0314,"[""Construct a rap battle, in the style of Epic...","[""[Zeus]\nYo, it's the king of the gods on the...","[""(Verse 1 - Zeus)\n\nI'm the king of the gods...",0,1,0,B
...,...,...,...,...,...,...,...,...,...,...
54598,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0,A
54599,4294692063,claude-2.0,llama-2-13b-chat,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0,A
54600,4294710549,claude-1,alpaca-13b,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0,A
54601,4294899228,palm-2,tulu-2-dpo-70b,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0,B


In [6]:
train.prompt.values[0]

'["What is the difference between marriage license and marriage certificate?","How can I get both of them as quick as possible in California ","What is the minimal time to get them? 1 day or 10 days?"]'

In [33]:
train.loc[724,'prompt']

'["relate philosophy to mathematics","Structure","when should structure be prioritized over concept","when should structure be prioritized over concept","teaching a structure, vs synthesizing the structure via specific concept learning"]'

In [26]:
output_as = []
output_bs = []
scuess_idx = []
for idx, row in tqdm(train.iterrows(), total=len(train)):
    try:
        prompt = row["prompt"]
        response_a = row["response_a"]
        response_b = row["response_b"]
        if prompt.find('","') != -1:
            cnt = prompt.count('","')
            prompt = row["prompt"].split('","')
            response_a = row["response_a"].split('","')
            response_b = row["response_b"].split('","')
            output_a = ""
            output_b = ""
            for sentence_idx in range(cnt + 1):
                if sentence_idx == 0:
                    output_a += "Prompt: " + prompt[0][2:].strip() + "\n"
                    output_a += "Response: " + response_a[0][2:].strip() + "\n\n"
                    output_b += "Prompt: " + prompt[0][2:].strip() + "\n"
                    output_b += "Response: " +response_b[0][2:].strip() + "\n\n"
                elif sentence_idx == cnt:
                    output_a += "Prompt: " + prompt[sentence_idx][:-2].strip() + "\n"
                    output_a += "Response: " + response_a[sentence_idx][:-2].strip() + "\n\n"
                    output_b += "Prompt: " + prompt[sentence_idx][:-2].strip() + "\n"
                    output_b += "Response: " + response_b[sentence_idx][:-2].strip() + "\n\n"
                else:
                    output_a += "Prompt: " + prompt[sentence_idx].strip() + "\n"
                    output_a += "Response: " + response_a[sentence_idx].strip() + "\n\n"
                    output_b += "Prompt: " + prompt[sentence_idx].strip() + "\n"
                    output_b += "Response: " + response_b[sentence_idx].strip() + "\n\n"
            output_as.append(output_a)
            output_bs.append(output_b)
        else:
            output_a = row["prompt"][2:-2].strip() + "\n" + row["response_a"][2:-2].strip()
            output_b = row["prompt"][2:-2].strip() + "\n" + row["response_b"][2:-2].strip()
            output_as.append(output_a)
            output_bs.append(output_b)
        scuess_idx.append(idx)
    except Exception as e:
        print(f"Error: {idx}")
        print(f"{e}")
        # output_as.append(" ")
        # output_bs.append(" ")

  5%|▍         | 2547/54603 [00:00<00:03, 13225.13it/s]

Error: 196
list index out of range
Error: 365
list index out of range
Error: 466
list index out of range
Error: 724
list index out of range
Error: 1674
list index out of range
Error: 2403
list index out of range
Error: 2515
list index out of range
Error: 2631
list index out of range
Error: 2839
list index out of range
Error: 2994
list index out of range


 13%|█▎        | 7276/54603 [00:00<00:03, 15243.48it/s]

Error: 3585
list index out of range
Error: 3753
list index out of range
Error: 3956
list index out of range
Error: 3987
list index out of range
Error: 4130
list index out of range
Error: 5120
list index out of range
Error: 6263
list index out of range
Error: 6612
list index out of range
Error: 6848
list index out of range
Error: 7303
list index out of range


 19%|█▉        | 10365/54603 [00:00<00:02, 15295.70it/s]

...

Error: 44889
list index out of range
Error: 45206
list index out of range
Error: 45355
list index out of range
Error: 45664
list index out of range
Error: 45758
list index out of range
Error: 46327
list index out of range
Error: 46647
list index out of range
Error: 46670
list index out of range
Error: 46695
list index out of range
Error: 46830
list index out of range
Error: 46930
list index out of range
Error: 47855
list index out of range


 92%|█████████▏| 50484/54603 [00:03<00:00, 14465.38it/s]

Error: 48889
list index out of range
Error: 49080
list index out of range
Error: 49244
list index out of range
Error: 49541
list index out of range
Error: 50534
list index out of range
Error: 50616
list index out of range
Error: 51382
list index out of range
Error: 51391
list index out of range


100%|██████████| 54603/54603 [00:03<00:00, 14678.19it/s]

Error: 52026
list index out of range
Error: 52184
list index out of range
Error: 52204
list index out of range
Error: 52615
list index out of range
Error: 53063
list index out of range
Error: 53116
list index out of range
Error: 53900
list index out of range
Error: 53905
list index out of range


In [4]:
assert len(scuess_idx)==len(output_as)

In [27]:
train = train.loc[scuess_idx,:].reset_index(drop = True)
train['instruction_a'] = output_as
train['instruction_b'] = output_bs

In [6]:
train = train.loc[:5,:]

In [58]:
def get_instruction_prompt(row):
    r_a = row.instruction_a
    r_b = row.instruction_b
    background = "Here are two question-answering dialogues. Compare two model performance on answering question, determine which is better."
    options = "###options\nA.Model A\nB. Model B\nC.Tie"
    #instruct_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{background}\n\n###Model A\n{r_a}###Model B\n{r_b}{options}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>"
    instruct_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nHere is background\n\n###Model A\nModel A res###Model B\nModel B res{options}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>A<|end_of_text|>"
    row['instruction_prompt'] = instruct_prompt
    return row

In [48]:
train = train.apply(lambda x: get_instruction_prompt(x), axis = 1)

In [49]:
print(train.loc[0,'instruction_prompt'])

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Here is background

###Model A
Model A res###Model B
Model B res###options
A.Model A
B. Model B
C.Tie
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>A<|end_of_text|>


In [46]:
tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id)

'<|end_of_text|>'

In [42]:
token_ids = tokenizer.encode(train.loc[0,'instruction_prompt'])
tokens = tokenizer.convert_ids_to_tokens(token_ids)
for token, token_id in zip(tokens, token_ids):
    print(f"Token: {token}, ID: {token_id}")

Token: <|begin_of_text|>, ID: 128000
Token: <|start_header_id|>, ID: 128006
Token: user, ID: 882
Token: <|end_header_id|>, ID: 128007
Token: ĊĊ, ID: 271
Token: Here, ID: 8586
Token: Ġis, ID: 374
Token: Ġbackground, ID: 4092
Token: ĊĊ, ID: 271
Token: ###, ID: 14711
Token: Model, ID: 1747
Token: ĠA, ID: 362
Token: Ċ, ID: 198
Token: Model, ID: 1747
Token: ĠA, ID: 362
Token: Ġres, ID: 594
Token: ###, ID: 14711
Token: Model, ID: 1747
Token: ĠB, ID: 426
Token: Ċ, ID: 198
Token: Model, ID: 1747
Token: ĠB, ID: 426
Token: Ġres, ID: 594
Token: ###, ID: 14711
Token: options, ID: 2945
Token: Ċ, ID: 198
Token: A, ID: 32
Token: .Model, ID: 5777
Token: ĠA, ID: 362
Token: Ċ, ID: 198
Token: B, ID: 33
Token: ., ID: 13
Token: ĠModel, ID: 5008
Token: ĠB, ID: 426
Token: Ċ, ID: 198
Token: C, ID: 34
Token: .T, ID: 844
Token: ie, ID: 648
Token: Ċ, ID: 198
Token: <|eot_id|>, ID: 128009
Token: Ċ, ID: 198
Token: <|start_header_id|>, ID: 128006
Token: assistant, ID: 78191
Token: <|end_header_id|>, ID: 128007


In [28]:
from torch.utils.data import Dataset
class InstructionDataSet(Dataset):
    def __init__(self, data, tokenizer, max_source_length, max_target_length):
        super(InstructionDataSet, self).__init__()
        #self.data = data.sample(len(data), random_state=0).reset_index(drop=True)
        self.data = data
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length
        # self.A_token = self.tokenizer.encode(text='A', add_special_tokens=False, truncation=True, )
        # self.B_token = self.tokenizer.encode(text='B', add_special_tokens=False, truncation=True, )
        # self.C_token = self.tokenizer.encode(text='C', add_special_tokens=False, truncation=True, )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        now_data = self.data.loc[index]
        r_a = now_data['instruction_a']
        r_b = now_data['instruction_b']
        background = "Here are two question-answering dialogues. Compare two model performance on answering question, determine which is better."
        options = "###options\nA.Model A\nB. Model B\nC.Tie"
        instruct_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nHere is {background}\n\n###Model A\n{r_a}###Model B\n{r_b}{options}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>A<|end_of_text|>"
        input_ids = self.tokenizer.encode(text=instruct_prompt, add_special_tokens=True, truncation=True,
                                          max_length=self.max_source_length)
        label = now_data['label']
        label_ids = self.tokenizer.encode(text=label, add_special_tokens=False, truncation=True, )
        input_ids = input_ids + label_ids + [self.tokenizer.eos_token_id]
        labels = [-100] * (len(input_ids) - 2) + label_ids + [self.tokenizer.eos_token_id]

        return {
            "input_ids": input_ids,
            "labels": labels
        }

In [29]:
from transformers import AutoTokenizer, AutoConfig
MODEL = "meta-llama/llama-3-transformers-8b-hf-v1"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:
from dataclasses import dataclass
@dataclass
class DataCollatorForInstruction:
     tokenizer: PreTrainedTokenizerBase
        model: Optional[Any] = None
        padding: Union[bool, str, PaddingStrategy] = True
        max_length: Optional[int] = None
        pad_to_multiple_of: Optional[int] = None
        label_pad_token_id: int = -100
        return_tensors: str = "pt"

    def __call__(self, features, return_tensors=None):
        if return_tensors is None:
            return_tensors = self.return_tensors
        labels = [feature["labels"] for feature in features] if "labels" in features[0].keys() else None
        # We have to pad the labels before calling `tokenizer.pad` as this method won't pad them and needs them of the
        # same length to return tensors.
        if labels is not None:
            max_label_length = max(len(l) for l in labels)
            if self.pad_to_multiple_of is not None:
                max_label_length = (
                        (max_label_length + self.pad_to_multiple_of - 1)
                        // self.pad_to_multiple_of
                        * self.pad_to_multiple_of
                )

            padding_side = self.tokenizer.padding_side
            for feature in features:
                remainder = [self.label_pad_token_id] * (max_label_length - len(feature["labels"]))
                if isinstance(feature["labels"], list):
                    feature["labels"] = (
                        feature["labels"] + remainder if padding_side == "right" else remainder + feature["labels"]
                    )
                elif padding_side == "right":
                    feature["labels"] = np.concatenate([feature["labels"], remainder]).astype(np.int64)
                else:
                    feature["labels"] = np.concatenate([remainder, feature["labels"]]).astype(np.int64)
        # breakpoint()
        features = self.tokenizer.pad(
            features,
            padding=True,
            max_length=max_label_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=return_tensors,
        )

        # prepare decoder_input_ids
        if (
                labels is not None
                and self.model is not None
                and hasattr(self.model, "prepare_decoder_input_ids_from_labels")
        ):
            decoder_input_ids = self.model.prepare_decoder_input_ids_from_labels(labels=features["labels"])
            features["decoder_input_ids"] = decoder_input_ids
        # breakpoint() # [(len(features[i]['input_ids']),len(features[i]['labels'])) for i in range(4)]
        return features

NameError: name 'PreTrainedTokenizerBase' is not defined

In [17]:
example = InstructionDataSet(train,tokenizer, 1024, 1)[0]

token_ids = example['input_ids']
tokens = tokenizer.convert_ids_to_tokens(token_ids)
for token, token_id in zip(tokens, token_ids):
    print(f"Token: {token}, ID: {token_id}")

Token: <|begin_of_text|>, ID: 128000
Token: <|start_header_id|>, ID: 128006
Token: user, ID: 882
Token: <|end_header_id|>, ID: 128007
Token: ĊĊ, ID: 271
Token: Here, ID: 8586
Token: Ġis, ID: 374
Token: ĠHere, ID: 5810
Token: Ġare, ID: 527
Token: Ġtwo, ID: 1403
Token: Ġquestion, ID: 3488
Token: -, ID: 12
Token: ans, ID: 598
Token: w, ID: 86
Token: ering, ID: 4776
Token: Ġdialog, ID: 7402
Token: ues, ID: 1157
Token: ., ID: 13
Token: ĠCompare, ID: 24702
Token: Ġtwo, ID: 1403
Token: Ġmodel, ID: 1646
Token: Ġperformance, ID: 5178
Token: Ġon, ID: 389
Token: Ġanswering, ID: 36864
Token: Ġquestion, ID: 3488
Token: ,, ID: 11
Token: Ġdetermine, ID: 8417
Token: Ġwhich, ID: 902
Token: Ġis, ID: 374
Token: Ġbetter, ID: 2731
Token: .ĊĊ, ID: 382
Token: ###, ID: 14711
Token: Model, ID: 1747
Token: ĠA, ID: 362
Token: Ċ, ID: 198
Token: Prompt, ID: 55715
Token: :, ID: 25
Token: ĠWhat, ID: 3639
Token: Ġis, ID: 374
Token: Ġthe, ID: 279
Token: Ġdifference, ID: 6811
Token: Ġbetween, ID: 1990
Token: Ġmarriage,